In [ ]:
import pandas as pd
import nltk
import pickle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import re

In [ ]:
movie_dataset = pd.read_csv('../input/imdb-movie-reviews-dataset/movie_data.csv')

In [ ]:
movie_dataset.head()

In [ ]:
count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [ ]:
print(count.vocabulary_)
print()
print(bag.toarray())

In [ ]:
np.set_printoptions(precision=2)
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

In [ ]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

In [ ]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf

In [ ]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

In [ ]:
movie_dataset.loc[0, 'review'][-50:]

In [ ]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

In [ ]:
preprocessor(movie_dataset.loc[0, 'review'][-50:])

In [ ]:
preprocessor("</a>This :) is :( a test :-)!")

In [ ]:
movie_dataset['review'] = movie_dataset['review'].apply(preprocessor)

In [ ]:
porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
tokenizer('runners like running and thus they run')

In [ ]:
tokenizer_porter('runners like running and thus they run')

In [ ]:
nltk.download('stopwords')

In [ ]:
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]

In [ ]:
X_train = movie_dataset.loc[:25000, 'review'].values
y_train = movie_dataset.loc[:25000, 'sentiment'].values
X_test = movie_dataset.loc[25000:, 'review'].values
y_test = movie_dataset.loc[25000:, 'sentiment'].values

In [ ]:
print("Review: ", X_train[1:2], "\n")
print("Sentiment: ", y_train[1])

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [ ]:
# filename = 'saved_model.sav'
# gs_lr_tfidf = pickle.load(open(filename, 'rb'))

In [ ]:
# clf = gs_lr_tfidf.best_estimator_
# print('Test Accuracy: %.3f' % clf.score(X_test, y_test))
# Test Accuracy: 0.90